In [1]:
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a DataFrame
df = pd.read_csv('aac_shelter_outcomes.csv')

# Drop the '_id' column from the DataFrame if it exists
if '_id' in df:
    df.drop(columns=['_id'], inplace=True)

# Dashboard Layout / View
app = Dash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
encoded_image = 'data:image/png;base64,{}'.format(encoded_image.decode())

# Define layout
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Img(src=encoded_image),  # HTML image tag with encoded logo image
    html.Div([
        # Add code for the interactive filtering options (e.g., dropdown)
        dcc.Dropdown(
            id='filter-type',  # Unique identifier for the dropdown component
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
            ],
            value='Water Rescue'  # Default value for the dropdown
        )
    ]),
    html.Hr(),
    # Set up the features for the interactive data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Number of rows per page
        sort_action='native',  # Enables sorting
        filter_action='native',  # Enables searching/filtering
        style_table={'overflowX': 'auto'},  # Enable horizontal scrolling
        style_cell={'textAlign': 'left'},  # Align cell text to the left
        style_data_conditional=[  # Apply conditional formatting to the table
            {
                'if': {'row_index': 'odd'},  # Apply style to odd rows
                'backgroundColor': 'rgb(248, 248, 248)'  # Set background color
            }
        ]
    ),
    html.Br(),
    html.Hr(),
    # Interaction Between Components / Controller
    # Callback to update the data table based on the selected filter
    html.Div(id='output-container'),
    html.Div(id='graph-container'),
    html.Div(id='map-container')
])

# Define callbacks
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Add code to filter interactive data table with MongoDB queries
    if filter_type == 'Water Rescue':
        query = {'rescue_type': 'Water Rescue'}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {'rescue_type': 'Mountain or Wilderness Rescue'}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {'rescue_type': 'Disaster or Individual Tracking'}
    else:
        query = {}  # No filter applied, retrieve all data

    filtered_data = df.query("rescue_type == @filter_type")  # Filter DataFrame based on rescue_type
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = filtered_data.to_dict('records')
    return (data, columns)

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-container', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is not None and len(viewData) > 0:
        df = pd.DataFrame(viewData)
        if 'breed' in df.columns:
            breed_counts = df['breed'].value_counts()
            fig = px.pie(names=breed_counts.index, values=breed_counts.values, title='Preferred Animals')
            return dcc.Graph(figure=fig)
        else:
            return html.Div("No 'breed' column found in the data")
    else:
        return html.Div("No data available for chart display")

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-container', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    ]

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)


[2024-02-25 19:22:32,821] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/computation/scope.py", line 231, in resolve
    return self.resolvers[key]
           ~~~~~~~~~~~~~~^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/collections/__init__.py", line 1006, in __getitem__
    return self.__missing__(key)            # support subclasses that define __missing__
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/collections/__init__.py", line 998, in __missing__
    raise KeyError(key)
KeyError: 'rescue_type'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/computation/scope.py", line 242, in resolve
    r

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
IndexError: list index out of range



Error on request:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/computation/scope.py", line 231, in resolve
    return self.resolvers[key]
           ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/collections/__init__.py", line 1006, in __getitem__
    return self.__missing__(key)            # support subclasses that define __missing__
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/collections/__init__.py", line 998, in __missing__
    raise KeyError(key)
KeyError: 'rescue_type'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/computation/scope.py", line 242, in resolve
    return self

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/computation/scope.py:231, in Scope.resolve(
    self=Scope(scope=['Timestamp',
 'datetime',
 'True',
... 'location_long',
 'age_upon_outcome_in_weeks']
),
    key='rescue_type',
    is_local=False
)
    230 if self.has_resolvers:
--> 231     return self.resolvers[key]
        self = Scope(scope=['Timestamp',
 'datetime',
 'True',
 'False',
 'list',
 'tuple',
 'inf',
 'Inf',
 '__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__builtin__',
 '__builtins__',
 '_ih',
 '_oh',
 '_dh',
 'In',
 'Out',
 'get_ipython',
 'exit',
 'quit',
 'open',
 '_',
 '__',
 '___',
 '__session__',
 '_i',
 '_ii',
 '_iii',
 '_i1',
 'Dash',
 'dl',
 'dcc',
 'html',
 'px',
 'dash_table',
 'Input',
 'Output',
 'State',
 'base64',
 'os',
 'np',
 'pd',
 'p